# This code based-on Kaggle editor
___


**Dataset:**
- https://www.kaggle.com/datasets/a2015003713/militaryaircraftdetectiondataset

**Reference:**
1. https://keras.io/api/applications/
2. https://keras.io/examples/vision/image_classification_from_scratch/
3. https://www.tensorflow.org/tutorials/keras/classification
4. https://github.com/AvinashNath2/Image-Classification-using-Keras/tree/master

`<3` **AvinashNath2**


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model

from sklearn.utils.class_weight import compute_class_weight

In [ ]:
"""
This script is used to build a simple ConvNet model for multi-class image classification.
based on the dataset of military aircraft detection.
using P100 GPU on Kaggle.
"""

# Image dimensions
img_width, img_height = 150, 150

# Define dataset directory
data_dir = '/kaggle/input/militaryaircraftdetectiondataset/crop'

# Preprocessing: used to rescale the pixel values from [0, 255] to [0, 1]
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)  # 40% for validation
batch_size = 32

# Automatically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',  # Assuming more than 2 classes
        subset='training')  # Use 70% for training

validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',  # Assuming more than 2 classes
        subset='validation')  # Use 30% for validation

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

class_weights_dict

# Build a simple ConvNet model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')  # Adjust for number of classes
])
# Compile the model
model.compile(loss='categorical_crossentropy',  # For multi-class classification
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Training
epochs = 50
train_samples = train_generator.samples
validation_samples = validation_generator.samples

result = model.fit(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator)

test_accuracy_string = str(test_accuracy).replace('.','')[:5]

model.save(f"afdet_{test_accuracy_string}.h5")
print("test_accuracy", test_accuracy_string)

In [ ]:
# plot graph from result.history
import matplotlib.pyplot as plt

# Plot Accuracy and Loss
def plot_training_history(history):
    # Plot training & validation accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training & validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function to plot the training history
plot_training_history(result)



In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reset the validation generator to ensure predictions start from the first batch
validation_generator.reset()

# Predict for the entire validation set
y_pred = model.predict(validation_generator, steps=len(validation_generator), verbose=1)

# Get true labels and predicted labels
y_true = validation_generator.classes  # True labels from the validation generator
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with highest probability

# Mapping class indices to class names
g_dict = validation_generator.class_indices
classes = list(g_dict.keys())

# Generate Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Function to plot the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap='Blues'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, 
                xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)
    plt.show()

# Plot the Confusion Matrix
plot_confusion_matrix(cm, classes, title='Confusion Matrix')

# Generate and print Classification Report
# print("Classification Report:")
# print(classification_report(y_true, y_pred_classes, target_names=classes))
